In NLP latest architecture is Transformer based architectures There's three types. Encoder-only models like BERT, decoder-only models like GPT and llama. There's encoder-decoder architectures as well: Encoder-decoder models can be implemented using various neural network architectures, and their names often reflect the type of network used or the specific application. Here are some key examples:
1. Transformer-based Encoder-Decoder Models:
These models are based on the Transformer architecture, which has significantly advanced the field of natural language processing.
T5 (Text-to-Text Transfer Transformer): T5 is a Transformer-based encoder-decoder model that treats all NLP tasks as text-to-text problems. It's widely used for tasks like machine translation, summarization, and question answering.
BART (Bidirectional and Auto-Regressive Transformer): BART is another Transformer-based encoder-decoder model that excels at denoising sequence-to-sequence tasks. It's particularly effective for text generation and summarization.
Pegasus (Pre-training with Extracted Gap-sentences for Abstractive Summarization): Pegasus is a Transformer-based encoder-decoder model specifically designed for abstractive summarization, where it generates summaries by focusing on key sentences extracted from the input text.
MT5 (Massively Multilingual Text-to-Text Transformer): MT5 is a multilingual variant of T5, trained on a large corpus of text in various languages.
FLAN-T5 (Scaling Instruction-Finetuned Language Models): FLAN-T5 is an extension of T5 that has been finetuned on a wide range of tasks and instructions to improve its generalization capabilities.
Code-T5: This is a variant of T5 designed specifically for code understanding and generation.
UL2 (Unifying Language Learning Paradigms): UL2 is another Transformer-based encoder-decoder model with a unified approach to language learning.
FLAN-UL2: This is a finetuned version of UL2 with improved performance on various tasks.
EdgeFormer: A Transformer-based encoder-decoder model designed for efficient seq2seq generation on devices with limited resources.
2. Models Utilizing Encoder-Decoder Architecture for Specific Tasks:
Encoder-decoder architectures can also be used as components within larger models designed for specific tasks:
VisionEncoderDecoderModel: This model initializes an image-to-text model with a pretrained vision model (like ViT) as the encoder and a pretrained language model (like BERT or GPT2) as the decoder. This allows it to perform tasks like image captioning and optical character recognition (OCR).
TrOCR (Transformer-based Optical Character Recognition): TrOCR is a specific instance of the VisionEncoderDecoderModel architecture, fine-tuned for OCR.
Note: Encoder-decoder architecture is a framework, and specific implementations can vary in their internal network structure (RNN, CNN, Transformer) and pre-training objectives.

LangChain can be used for app development. Web-side of app development is still done with FastAPI. If you want to include agents then Langgraph is used. Fine-tuning of the model for best hyperparameters can be done using LoRA or QLoRA. LoRA is where memory is a constraint but want to maintain high precision. QLoRA (quantized lora) is used to optimize memory efficiency in comprise for a minimal loss in performance.

Llama 2 and 3 and mistral ai were attempted but did not work with system ram and gpu usage constraints. Tiny Llama was finally leveraged on a dataset that contained two columns: one with a full clinical note and the other with the summary written by a healthcare provider. The llm was given the task to summarize a given synthetic clinical note by chatgpt4. The result was an accurate response with extra unnecessary details

In [1]:
!pip install triton==2.1.0 bitsandbytes==0.41.0 peft==0.7.0 transformers==4.38.2 accelerate==0.30.0 trl==0.4.7

ERROR: Could not find a version that satisfies the requirement triton==2.1.0 (from versions: none)
ERROR: No matching distribution found for triton==2.1.0

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install huggingface_hub
!pip install numpy
import numpy as np


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 5.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [5]:
#!pip install triton
!pip install trl
import torch
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)
from transformers.generation import LogitsProcessorList, TopKLogitsWarper, TopPLogitsWarper
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


Llama model requires too much system ram and is crashing which is the reason for using tiny llama.

In [6]:
llama_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    # Remove quantization_config
).to('cpu') # Explicitly move the model to CPU
llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

The below code is specifically for GPU. Bitsandbytes config with quantization is for gpu. The above code is for CPU.

In [ ]:
###llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path= "aboonaji/llama2finetune-v3",
                                                  # quantization_config= BitsAndBytesConfig(load_in_4bit = True,
                                                   # bnb_4bit_compute_dtype = getattr(torch, "float16"), bnb_4bit_quant_type = "nf4"))
##llama_model.config.use_cache = False
##llama_model.config.pretraining_tp = 1

In [7]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path= "TinyLlama/TinyLlama-1.1B-Chat-v1.0", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

In [8]:
training_arguments = TrainingArguments(output_dir= "./results", per_device_train_batch_size = 1, max_steps = 4)

In [9]:
!pip install -U datasets huggingface_hub fsspec


load_dataset("geekdom/clinical_data")
print(load_dataset)

  Using cached huggingface_hub-1.2.3-py3-none-any.whl.metadata (13 kB)
  Using cached fsspec-2025.12.0-py3-none-any.whl.metadata (10 kB)
Using cached huggingface_hub-1.2.3-py3-none-any.whl (520 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.3 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.2.3 which is incompatible.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.2.3 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


Generating train split: 100%|██████████| 19756/19756 [00:00<00:00, 90022.13 examples/s]

<function load_dataset at 0x13291e700>


In [31]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load dataset
dataset = load_dataset("geekdom/clinical_data", split="train")

# Format for SFT / text-to-text
def format_for_sft(example):
    return {
        "text": f"Summarize:\n{example['prompt']}\n\nSummary: {example['response']}"
    }

dataset = dataset.map(format_for_sft)

# Keep only the 'text' column
dataset = dataset.remove_columns([col for col in dataset.column_names if col != "text"])

# Load tokenizer for your model
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")  # replace with your LLaMA path

# Tokenize dataset
def tokenize_fn(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_fn, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])


Map: 100%|██████████| 19756/19756 [00:02<00:00, 7380.32 examples/s]


In [32]:
tokenized_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 19756
})

In [13]:
!pip install --upgrade peft

  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
Using cached huggingface_hub-0.36.0-py3-none-any.whl (566 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface_hub 1.2.3
    Uninstalling huggingface_hub-1.2.3:
      Successfully uninstalled huggingface_hub-1.2.3

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [27]:
from transformers import Trainer

In [34]:
from transformers import Trainer, DataCollatorForLanguageModeling

# Use a data collator for causal language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set False for causal LM
)

# Make labels same as input_ids for causal LM
def add_labels(batch):
    batch["labels"] = batch["input_ids"].clone()
    return batch

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)

# Initialize Trainer
trainer = Trainer(
    model=llama_model,
    args=training_arguments,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator)

Map: 100%|██████████| 19756/19756 [00:00<00:00, 84667.96 examples/s]
/var/folders/ht/stt2pwyj30nd38vj2cm2nbzc0000gn/T/ipykernel_21439/4241640021.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [35]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
/Users/saiashutoshchellarapu/.pyenv/versions/3.12.3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Insufficient Memory (00000008:kIOGPUCommandBufferCallbackErrorOutOfMemory)
	<AGXG17GFamilyCommandBuffer: 0x7c027d180>
    label = <none> 
    device = <AGXG17GDevice: 0x7d85a0000>
        name = Apple M5 
    commandQueue = <AGXG17GFamilyCommandQueue: 0x7d85a4000>
        label = <none> 
        device = <AGXG17GDevice: 0x7d85a0000>
            name = Apple M5 
    retainedReferences = 1
Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Insufficient Memory (00000008:kIOGPUCommandBufferCallbackErrorOutOfMemory)
	<AGXG17GFamilyCommandBuffer: 0x7bf6d3b80>
    label = <none> 
    device = <AGXG17GDevice: 0x7d85a0000>
        name = Apple M5 
    commandQueue = <AGXG17GFamilyCommandQueue: 0x7d85a4000>
        label = <none> 
        device = <AGXG17GDev

TrainOutput(global_step=4, training_loss=0.9194257259368896, metrics={'train_runtime': 230.5088, 'train_samples_per_second': 0.017, 'train_steps_per_second': 0.017, 'total_flos': 12712088174592.0, 'train_loss': 0.9194257259368896, 'epoch': 0.00020247013565499088})

In [39]:
import torch

# Select device: MPS if available, else CPU
device = "mps" if torch.backends.mps.is_available() else "cpu"
llama_model.to(device)

user_prompt = (
    """Below is an instruction that describes a task. Write a response that appropriately completes the request.\n
    Make sure to use line breaks when appropriate.\n
    ### Instruction:\n
    Below is a clinical note.\n
    Your task is to summarize the clinical note in that you describe the patient's course of progression.\n
    ### Input:\n
    Hospital Course: The patient was admitted to the ICU six days after testing positive for COVID-19 due to worsening respiratory distress.\n
    Early physical therapy and prone positioning led to improved oxygen saturation from 85% to 94%. After five days of supportive care and rehabilitation,\n
    the patient was transferred to the general ward and continued progressing with assisted ambulation and breathing exercises.\n
    Discharge Condition: At discharge, the patient was stable, breathing comfortably on room air, and able to walk short distances with minimal assistance.\n
    Oxygen saturation and respiratory rate were within normal limits.\n
    ### Response:"""
)

# Tokenize and move inputs to same device
inputs = llama_tokenizer(user_prompt, return_tensors="pt", padding=True, truncation=True)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Move model to float32 to avoid NaNs on MPS
llama_model = llama_model.to(torch.float32)

# Disable sampling to avoid NaNs (use greedy generation)
with torch.no_grad():
    outputs = llama_model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=False,  # <--- Important: Greedy avoids NaNs
        temperature=1.0,  # ignored with do_sample=False
        top_p=1.0,        # ignored with do_sample=False
        pad_token_id=llama_tokenizer.pad_token_id
    )

# Decode output
response = llama_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Make sure to use line breaks when appropriate.

    ### Instruction:

    Below is a clinical note.

    Your task is to summarize the clinical note in that you describe the patient's course of progression.

    ### Input:

    Hospital Course: The patient was admitted to the ICU six days after testing positive for COVID-19 due to worsening respiratory distress.

    Early physical therapy and prone positioning led to improved oxygen saturation from 85% to 94%. After five days of supportive care and rehabilitation,

    the patient was transferred to the general ward and continued progressing with assisted ambulation and breathing exercises.

    Discharge Condition: At discharge, the patient was stable, breathing comfortably on room air, and able to walk short distances with minimal assistance.

    Oxygen saturation and respiratory rate were within normal limits.

    ### Re

# Result:

Below is the input given to the model. The model produces a decent answer but the answer depends exremely on the instructions given. The answer is accurate but lacks precision because extra details before or after the response are sometimes given. The response is unique each generation of response.

# Given Input

Hospital Course:

The patient was admitted to the ICU six days after testing positive for COVID-19 due to worsening respiratory distress. Early physical therapy and prone positioning led to improved oxygen saturation from 85% to 94%. After five days of supportive care and rehabilitation, the patient was transferred to the general ward and continued progressing with assisted ambulation and breathing exercises.

Discharge Condition:

At discharge, the patient was stable, breathing comfortably on room air, and able to walk short distances with minimal assistance. Oxygen saturation and respiratory rate were within normal limits.